In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.functions import array_intersect

# Iniciar uma sessão Spark
spark = SparkSession.builder \
    .appName("Read Parquet File") \
    .getOrCreate()

# Substitua 'caminho/do/arquivo.parquet' pelo caminho real do seu arquivo .parquet
file_path1 = 'part-00000-b5473193-ba27-4dc1-aa69-ec770eb7ef82-c000.snappy.parquet'

file_path2 = 'part-00000-f43e1eee-9720-4ebb-947c-626c9e1ba2f1-c000.snappy.parquet'

# Carregar o arquivo .parquet para um DataFrame
json = spark.read.parquet(file_path1)
csv = spark.read.parquet(file_path2)

csv = csv.filter(csv["genero"].contains("Drama") & (csv["anoLancamento"] >= 2000))

# Extrair os nomes dos gêneros como uma lista de strings
json = json.withColumn("genre_names", expr("transform(genres, x -> x.name)"))

# Filtrar os filmes de drama desde 2014 no DataFrame json
json = json.filter(array_contains(json["genre_names"], "Drama") & (year("release_date") >= 2000))


joined_df = json.join(csv, json["imdb_id"] == csv["id"], "inner")

filmes = joined_df.select(
    json["imdb_id"],
    csv["tituloPrincipal"],
    json["production_countries"],
    json["release_date"],
    json["title"],
)
popularidade = joined_df.select(
    json["imdb_id"],
    csv["notaMedia"],
    csv["numeroVotos"],
    json["vote_average"],
    json["vote_count"],
)

financeiro = joined_df.select(
    json["imdb_id"],
    json["budget"],
    json["revenue"],
)

outros = joined_df.select(
    json["imdb_id"],
    json["genres"],
    json["original_language"],
    json["overview"],
    json["popularity"],
    json["runtime"],
    json["status"],
)

# Contar o número de linhas no DataFrame csv
count_csv = csv.count()
print("Número de linhas no DataFrame csv:", count_csv)

# Filtrar os filmes do gênero "Drama"
drama_movies = csv.filter(csv["genero"].contains("Drama"))

# Contar o número de filmes do gênero "Drama"
count_drama_movies = drama_movies.count()
print("Número de filmes do gênero Drama:", count_drama_movies)

# Contar o número de linhas no DataFrame json
count_json = json.count()
print("Número de linhas no DataFrame json:", count_json)

# Extrair os nomes dos gêneros como uma lista de strings
json = json.withColumn("genre_names", expr("transform(genres, x -> x.name)"))

# Contar o número de filmes do gênero Drama
count_drama_movies = json.filter(array_contains(json["genre_names"], "Drama")).count()

# Mostrar o resultado
print("Número de filmes do gênero Drama:", count_drama_movies)

# Converter a coluna release_date para o tipo Date
filmes = filmes.withColumn("release_date", to_date("release_date", "yyyy-MM-dd"))

# Extrair o ano da data
filmes = filmes.withColumn("year", year("release_date"))

# Contar o número de filmes por ano
movies_per_year = filmes.groupBy("year").count().orderBy("year")

# Mostrar o resultado
movies_per_year.show(1000)


america_do_sul_codigos = ['AR', 'BO', 'BR', 'CL', 'CO', 'EC', 'FK', 'GF', 'GY', 'PY', 'PE', 'SR', 'UY', 'VE']

# Filtrar os filmes da América Latina
latin_america_movies = csv.join(json, csv["id"] == json["imdb_id"], "inner") \
                           .filter(array_intersect(array([lit(code) for code in america_do_sul_codigos]), col("production_countries.iso_3166_1")).getItem(0).isNotNull())

# Explodir a lista de países em múltiplas linhas
latin_america_movies_exploded = latin_america_movies.withColumn("country", explode("production_countries.name"))

# Contar o número total de filmes da América Latina
total_movies_latin_america = latin_america_movies.count()
print("Número total de filmes da América Latina:", total_movies_latin_america)

# Contar o número de filmes por país da América Latina
movies_per_country = latin_america_movies_exploded.groupBy("country").count()
movies_per_country.show(1000)

# Finalizar a sessão Spark
spark.stop()


Número de linhas no DataFrame csv: 64327
Número de filmes do gênero Drama: 64327
Número de linhas no DataFrame json: 42693
Número de filmes do gênero Drama: 42693
+----+-----+
|year|count|
+----+-----+
|2000|  861|
|2001| 1048|
|2002| 1093|
|2003| 1095|
|2004| 1177|
|2005| 1285|
|2006| 1427|
|2007| 1514|
|2008| 1591|
|2009| 1716|
|2010| 1722|
|2011| 1873|
|2012| 1951|
|2013| 2242|
|2014| 2332|
|2015| 2441|
|2016| 2532|
|2017| 2670|
|2018| 2756|
|2019| 2846|
|2020| 1994|
|2021| 2226|
|2022| 2030|
|2023|  258|
|2024|   13|
+----+-----+

Número total de filmes da América Latina: 2320
+--------------------+-----+
|             country|count|
+--------------------+-----+
|                Chad|    1|
|            Paraguay|   18|
|              Russia|    5|
|              Sweden|    7|
|              Guyana|    2|
|         Philippines|    1|
|            Malaysia|    1|
|           Singapore|    1|
|                Iraq|    1|
|             Germany|  111|
|         Afghanistan|    1|
|     

In [18]:
import pandas as pd

# Carregar os dados do arquivo Parquet para um DataFrame do Pandas
df = pd.read_parquet('part-00000-456f3bb8-3051-4b08-a529-4c4f5f2e85ee-c000.snappy.parquet')

# Lista de códigos de países da América do Sul
america_do_sul_codigos = ['AR', 'BO', 'BR', 'CL', 'CO', 'EC', 'FK', 'GF', 'GY', 'PY', 'PE', 'SR', 'UY', 'VE']

# Filtrar os filmes da América do Sul
filmes_sul_america = df[df['production_countries'].apply(lambda x: any(country['iso_3166_1'] in america_do_sul_codigos for country in x))]

# Contar o número total de filmes da América do Sul
total_filmes_sul_america = len(filmes_sul_america)

# Imprimir o total de filmes da América do Sul
print("Total de filmes da América do Sul:", total_filmes_sul_america)

# Ordenar os filmes da América do Sul por popularidade
filmes_sul_america.sort_values(by='vote_average', ascending=False, inplace=True)

# Imprimir os top 100 filmes da América do Sul
print("\nTop 100 filmes da América do Sul:")
for i, (_, filme) in enumerate(filmes_sul_america.head(100).iterrows(), 1):
    print(f"{i}. {filme['title']} ({filme['vote_average']}) - {', '.join([country['name'] for country in filme['production_countries']])}")

# Contar quantos filmes de cada país da América do Sul estão no top 100
country_counts = {}
for _, filme in filmes_sul_america.head(100).iterrows():
    for country in filme['production_countries']:
        if country['iso_3166_1'] in america_do_sul_codigos:
            country_counts[country['name']] = country_counts.get(country['name'], 0) + 1

# Apresentar o número de filmes por país no top 100 da América do Sul
print("\nNúmero de filmes por país no top 100 da América do Sul:")
for country_name, count in country_counts.items():
    print(f"{country_name}: {count}")

# Imprimir o top 10 filmes de cada país da América do Sul
for country_name in country_counts.keys():
    country_movies = filmes_sul_america[filmes_sul_america['production_countries'].apply(lambda x: any(country['name'] == country_name for country in x))]
    country_movies.sort_values(by='vote_average', ascending=False, inplace=True)
    print(f"\nTop 10 filmes de {country_name}:")
    for i, (_, filme) in enumerate(country_movies.head(10).iterrows(), 1):
        print(f"{i}. {filme['title']} ({filme['vote_average']})")


Total de filmes da América do Sul: 1442

Top 100 filmes da América do Sul:
1. Ana. Sem Título (10.0) - Brazil
2. Vestigio (10.0) - Argentina
3. Saber crecer (10.0) - Paraguay
4. Boi de Lágrimas (10.0) - Brazil
5. Salamandra (10.0) - Belgium, Brazil, France, Germany, United Kingdom
6. Las mujeres deciden (10.0) - Ecuador
7. Um Filme Francês (10.0) - Brazil
8. Theo, além da liberdade (10.0) - Brazil
9. Tiempo Suspendido (10.0) - Argentina, Mexico
10. Ojalá vivas tiempos interesantes (10.0) - Argentina
11. Extirpador de Idolatrias (10.0) - Peru
12. Verlust (10.0) - Brazil, Uruguay
13. Tierra yerma (10.0) - United Kingdom, Chile, France
14. Norte (10.0) - Peru
15. A Praia do Fim do Mundo (10.0) - Brazil
16. Esencial (10.0) - Argentina
17. Perros sin cola (10.0) - Chile
18. Histórias de Alice (10.0) - Brazil, Portugal
19. Metade do Céu (10.0) - South Africa, Brazil, China, India, Russia
20. Los del suelo (9.5) - Argentina
21. Helen (9.5) - Brazil
22. Pule ou Ocorrência às 4h48 (9.3) - Brazi

/var/folders/5q/4krf7jmj6fgfll_b9kgmff9r0000gn/T/ipykernel_3350/3585658541.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filmes_sul_america.sort_values(by='vote_average', ascending=False, inplace=True)
/var/folders/5q/4krf7jmj6fgfll_b9kgmff9r0000gn/T/ipykernel_3350/3585658541.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country_movies.sort_values(by='vote_average', ascending=False, inplace=True)


In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.functions import array_intersect

# Iniciar uma sessão Spark
spark = SparkSession.builder \
    .appName("Read Parquet File") \
    .getOrCreate()

# Substitua 'caminho/do/arquivo.parquet' pelo caminho real do seu arquivo .parquet
file_path1 = 'part-00000-5518d4f0-6f6a-49f9-aa09-b9f6f7063487-c000.snappy.parquet'

file_path2 = 'part-00000-a5c57ffd-8007-4697-b57f-b6085cf297a6-c000.snappy.parquet'

file_path3 = 'part-00000-477c9d7b-713e-47fb-8422-aa5072c62c02-c000.snappy.parquet'

file_path4 = 'part-00000-a43f265a-77d7-416b-a9e2-14af279fba47-c000.snappy.parquet'


# Carregar o arquivo .parquet para um DataFrame
filmes = spark.read.parquet(file_path1)
financeiro = spark.read.parquet(file_path2)
popularidade = spark.read.parquet(file_path3)
outros = spark.read.parquet(file_path4)

filmes.printSchema()
filmes.show()
financeiro.printSchema()
financeiro.show()
popularidade.printSchema()
popularidade.show()
outros.printSchema()
outros.show()

# Finalizar a sessão Spark
spark.stop()


root
 |-- id: string (nullable = true)
 |-- tituloCSV: string (nullable = true)
 |-- origem: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- iso_3166_1: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |-- lancamento: date (nullable = true)
 |-- tituloJSON: string (nullable = true)

+---------+--------------------+--------------------+----------+--------------------+
|       id|           tituloCSV|              origem|lancamento|          tituloJSON|
+---------+--------------------+--------------------+----------+--------------------+
|tt0116391|                Gang|       [{IN, India}]|2000-04-14|                Gang|
|tt0116748|Karobaar: The Bus...|                  []|2000-09-15|            Karobaar|
|tt0118589|             Glitter|[{US, United Stat...|2001-09-21|Glitter: O Brilho...|
|tt0118694|In the Mood for Love|[{FR, France}, {H...|2000-09-29| Amor à Flor da Pele|
|tt0118926| The Dancer Upstairs|[{ES, Spain}, {US...